In [2]:
import mediapipe as mp
import numpy as np
import cv2 as cv

In [14]:
cap = cv.VideoCapture(0)
name = input("Enter the name of the data")
holistic = mp.solutions.holistic
hands = mp.solutions.hands
pose = mp.solutions.pose
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

X = []
data_size=0
while True:
    lst = []
    _,fm = cap.read()
    
    fm = cv.flip(fm,1)
    res = holis.process(cv.cvtColor(fm,cv.COLOR_BGR2RGB))
#     if res.face_landmarks:
#         for i in range(len(res.face_landmarks.landmark)):
#             lst.append(i.x - res.face_landmarks.landmark.x)
#             lst.append(i.y - res.face_landmarks.landmark.y)
#     if res.pose_landmarks:
#         for i in range(ln(res.pose_landmarks.landmark)):
#             lst.append(i.x - res.face_landmarks.landmark.x)
#             lst.append(i.y - res.face_landmarks.landmark.y)
            
    if res.left_hand_landmarks:
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:
        for i in range(42):
            lst.append(0.0)
            
    if res.right_hand_landmarks:
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        for i in range(42):
            lst.append(0.0)
    X.append(lst)
    data_size = data_size + 1
    
    res = holis.process(cv.cvtColor(fm,cv.COLOR_BGR2RGB))
    
#     drawing.draw_landmarks(fm,res.pose_landmarks,holistic.POSE_CONNECTIONS)
#     drawing.draw_landmarks(fm,res.face_landmarks,holistic.FACEMESH_CONTOURS)
    drawing.draw_landmarks(fm,res.left_hand_landmarks,holistic.HAND_CONNECTIONS)
    drawing.draw_landmarks(fm,res.right_hand_landmarks,holistic.HAND_CONNECTIONS)
    
    
    cv.putText(fm,str(data_size),(50,50),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    
    cv.imshow('window',fm)
    
    if cv.waitKey(1)==27 or  data_size > 499:
        cv.destroyAllWindows()
        cv.waitKey(1)
        cap.release()
        break
np.save(f"data/{name}.npy",np.array(X))
print(np.array(X).shape)

Asked for all format descriptions...
[setting] Setting kCMIODevicePropertyDeviceMaster to 6846
[setting] Setting kCMIOStreamPropertyFormatDescription
Got new format : 1920x1080
Asked for all format descriptions...
[setting] Setting kCMIODevicePropertyDeviceMaster to -1
[setting] Setting kCMIODevicePropertyDeviceMaster to 6846
[setting] Setting kCMIODevicePropertyDeviceMaster to -1
Asked for all format descriptions...
Enter the name of the dataZ
[RemoteFrameSource] Releasing RemoteFrameSource


FileNotFoundError: [Errno 2] No such file or directory: 'data/Z.npy'

## Importing Modules

## Data Importation and Preprocessing

In [17]:
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,rotation_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)


## Training and Validation Split

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

## Model Building

In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])


## Model Training

In [ ]:
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch = 28709 // 64,
    epochs=75,
    validation_data = val_generator,
    validation_steps = 7178 // 64
)

## Model saving

In [ ]:
emotion_model.save_weights('model.h5')


In [24]:
from tensorflow.keras.models import load_model

model = load_model('model.h5')

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7fda4af436d0>.

In [20]:
import h5py

f = h5py.File('model.h5', mode='r')


In [22]:
f.summary()

AttributeError: 'File' object has no attribute 'summary'

In [39]:
%tensorboard --logdir save


ModuleNotFoundError: No module named 'tensorboard --logdir="model'